In [ ]:
!pip install -U langchain-core langchain-huggingface langchain-chroma chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 138.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.1 MB/

In [ ]:
import json
import os
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


In [ ]:
embeddings_Model = HuggingFaceEmbeddings(
    model_name='bkai-foundation-models/vietnamese-bi-encoder',
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [ ]:
law_types = ['GTVT', 'GiaoDuc', 'HonNhanGiaDinh', 'LaoDongTienLuong', 'TrachNhiemHinhSu']
input_folder = "/content/drive/MyDrive/NLP/Đô án/Data_with_metadata/"
db_folder_path = "/content/drive/MyDrive/NLP/Đô án/Vector Database"

all_documents = []

for law in law_types:
    file_path = os.path.join(input_folder, f"Data_with_Metadata_{law}.json")
    if not os.path.exists(file_path):
        continue

    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for item in data:
        page_content = item.pop("SemanticChunk-Content", "")

        clean_metadata = {}
        for key, value in item.items():
            if isinstance(value, list):
                clean_metadata[key] = ", ".join([str(v) for v in value])
            else:
                clean_metadata[key] = value
        doc = Document(page_content=page_content, metadata=clean_metadata)
        all_documents.append(doc)

print(f"Đã chuẩn bị xong {len(all_documents)} đoạn văn bản.")

Đã chuẩn bị xong 15657 đoạn văn bản.


In [ ]:
print("Đang tiến hành nạp vào ChromaDB (Vui lòng đợi)...")

vector_db = Chroma.from_documents(
    documents=all_documents,
    embedding=embeddings_Model,
    persist_directory=db_folder_path
)

print(f"THÀNH CÔNG! Toàn bộ dữ liệu đã được lưu tại thư mục: {db_folder_path}")

Đang tiến hành nạp vào ChromaDB (Vui lòng đợi)...
THÀNH CÔNG! Toàn bộ dữ liệu đã được lưu tại thư mục: /content/drive/MyDrive/NLP/Đô án/Vector Database


In [ ]:
query = "Vượt đèn đỏ sẽ bị phạt bao nhiêu tiền"
results = vector_db.similarity_search(query, k=1)

if results:
    print("Nội dung tìm thấy:", results[0])

Nội dung tìm thấy: page_content='1. Phạt tiền từ 200.000 đồng đến 400.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây:
a) Không chấp hành hiệu lệnh, chỉ dẫn của biển báo hiệu, vạch kẻ đường, trừ các hành vi vi phạm quy định tại điểm b, điểm d, điểm e khoản 2; điểm a, điểm c, điểm d, điểm h khoản 3; điểm a, điểm b, điểm c, điểm d khoản 4; điểm b, điểm d khoản 6; điểm a, điểm b, điểm c khoản 7; điểm a khoản 8; điểm b khoản 9; điểm a khoản 10 Điều này;
b) Không có tín hiệu trước khi vượt hoặc có tín hiệu vượt xe nhưng không sử dụng trong suốt quá trình vượt xe;
c) Lùi xe mô tô ba bánh không quan sát hai bên, phía sau xe hoặc không có tín hiệu lùi xe;
d) Chở người ngồi trên xe sử dụng ô (dù);
đ) Không tuân thủ các quy định về nhường đường tại nơi đường giao nhau, trừ các hành vi vi phạm quy định tại điểm c, điểm d khoản 6 Điều này;
e) Chuyển làn đường không đúng nơi cho phép hoặc không có tín hiệu báo trước hoặc chuyển làn đường không đúng quy định “mỗi